In [1]:
import pandas as pd
import re

In [2]:
alletf = pd.read_csv('C:/Users/jxjsj/Desktop/alletf.csv')
alletf['Asset Class'] = alletf['Asset Class'].apply(lambda x: 'Other' if x in ['Alternatives', 'Multi-Asset', 'Real Estate', 'Volatility'] else x)

In [3]:
# 提取杠杆与反向ETF
Letf = alletf[(alletf.Leveraged != 'No') & (alletf.Inverse == 'No')].rename(columns={'Total Assets ':'Total Assets'})
Letf['Total Assets'] = Letf['Total Assets'].apply(lambda x: float(x[1:].replace(',','')))

Ietf = alletf[alletf.Inverse != 'No'].rename(columns={'Total Assets ':'Total Assets'})
Ietf['Total Assets'] = Ietf['Total Assets'].apply(lambda x: float(x[1:].replace(',','')))
Ietf['Leveraged'] = Ietf['Leveraged'].apply(lambda x: '-1x' if x == ';No' else '-'+x)

In [4]:
LandIetf = pd.concat([Letf,Ietf],axis=0)

In [5]:
tolSizeOfLandI = LandIetf['Total Assets'].sum()
tolCountOfLandI = LandIetf['Total Assets'].count()

In [63]:
print(len(Letf),len(Ietf),len(alletf))

150 124 2291


In [62]:
print(Letf.Inverse.unique())
print(Letf.Leveraged.unique())

print(Ietf.Inverse.unique())
print(Ietf.Leveraged.unique())

['No']
['3x' '2x' '1.5x' '1.25x' '4x']
['Yes']
['No' '2x' '3x' '0.5x']


In [6]:
# 储存器
save_dict = {}

In [ ]:
'''
杠杆
'''

In [7]:
for groupkind in [['Leveraged'],['Leveraged','Asset Class'],['Asset Class']]:
    d1=Letf.groupby(groupkind).count()[['Symbol']].reset_index().rename(columns={'Symbol':'Count'})
    d2=Letf.groupby(groupkind).sum()[['Total Assets']].reset_index().rename(columns={'Total Assets':'TolSize'})
    d3=Letf.groupby(groupkind).mean()[['Total Assets']].reset_index().rename(columns={'Total Assets':'AvgSize'})
    d=d1.merge(d2,on=groupkind)
    d=d.merge(d3,on=groupkind)
    d['TolSizeRatio']=d['TolSize']/tolSizeOfLandI
    d['TolCountRatio']=d['Count']/tolCountOfLandI
    name=''.join(groupkind)
    save_dict['L'+name] = d
#     d.to_csv('C:/Users/jxjsj/Desktop/东吴金工研报/ETFusData/L'+name+'.csv',index=False)

In [ ]:
'''
反向
'''

In [8]:
for groupkind in [['Leveraged'],['Leveraged','Asset Class'],['Asset Class']]:
    d1=Ietf.groupby(groupkind).count()[['Symbol']].reset_index().rename(columns={'Symbol':'Count'})
    d2=Ietf.groupby(groupkind).sum()[['Total Assets']].reset_index().rename(columns={'Total Assets':'TolSize'})
    d3=Ietf.groupby(groupkind).mean()[['Total Assets']].reset_index().rename(columns={'Total Assets':'AvgSize'})
    d=d1.merge(d2,on=groupkind)
    d=d.merge(d3,on=groupkind)
    d['TolSizeRatio']=d['TolSize']/tolSizeOfLandI
    d['TolCountRatio']=d['Count']/tolCountOfLandI
    name=''.join(groupkind)
    save_dict['I'+name] = d
#     d.to_csv('C:/Users/jxjsj/Desktop/东吴金工研报/ETFusData/I'+name+'.csv',index=False)

In [ ]:
'''
杠杆和反向
'''

In [9]:
d1=LandIetf.groupby(groupkind).count()[['Symbol']].reset_index().rename(columns={'Symbol':'Count'})
d2=LandIetf.groupby(groupkind).sum()[['Total Assets']].reset_index().rename(columns={'Total Assets':'TolSize'})
d3=LandIetf.groupby(groupkind).mean()[['Total Assets']].reset_index().rename(columns={'Total Assets':'AvgSize'})
d=d1.merge(d2,on=groupkind)
d=d.merge(d3,on=groupkind)
d['TolSizeRatio']=d['TolSize']/tolSizeOfLandI
d['TolCountRatio']=d['Count']/tolCountOfLandI
name=''.join(groupkind)
save_dict['LandI'+name] = d
# d.to_csv('C:/Users/jxjsj/Desktop/东吴金工研报/ETFusData/LandI'+name+'.csv',index=False)

In [ ]:
'''
杠/反总规模
'''

In [10]:
Letf.sum()[['Total Assets']]

Total Assets    3.72048e+10
dtype: object

In [11]:
Ietf.sum()[['Total Assets']]

Total Assets    1.2122e+10
dtype: object

In [12]:
LandIetf.sum()[['Total Assets']]

Total Assets    4.93268e+10
dtype: object

In [ ]:
'''
输出
'''

In [13]:
# 输入一个键值对为 sheet名-sheet内容（字符串，DataFrame）的字典输出一个excel
def stock_to_excel(outputpath,df_dict):
        """
        outputpath: str  输出路径，例如'c:/XXX/YYYYY/'
        df_dict   : dict stock_inf_get函数得到的结果字典
        """
        outputfile = outputpath + 'PlotData.xlsx'
        writer = pd.ExcelWriter(outputfile)
        for key, value in df_dict.items():
            value.to_excel(writer,key,index=False)
        writer.save()  

In [14]:
stock_to_excel('C:/Users/jxjsj/Desktop/东吴金工研报/ETFusData/',save_dict)

In [37]:
Letf.columns

Index(['Symbol', 'ETF Name', 'Asset Class', 'Total Assets ', 'YTD',
       'Avg Volume', 'Previous Closing Price', '1-Day Change', 'Inverse',
       'Leveraged', '1 Week', '4 Week', '1 year', '3 year', '5 year', 'YTD FF',
       '1 Week FF', '4 Week FF', '1 Year FF', '3 Year FF', '5 Year FF',
       'ETFdb.com Category', 'Inception', 'ER', 'Commission Free',
       'Annual Dividend Rate', 'Dividend Date', 'Dividend',
       'Annual Dividend Yield %', 'P/E Ratio', 'Beta', '# of Holdings',
       '% In Top 10', 'ST Cap Gain Rate', 'LT Cap Gain Rate', 'Tax Form',
       'Lower Bollinger', 'Upper Bollinger', 'Support 1', 'Resistance 1',
       'RSI', 'Overall Rating', 'Liquidity Rating', 'Expenses Rating',
       'Returns Rating', 'Volatility Rating', 'Dividend Rating',
       'Concentration Rating', 'ESG Score', 'ESG Score Peer Percentile (%)',
       'ESG Score Global Percentile (%)',
       'Carbon Intensity (Tons of CO2e / $M Sales)',
       'SRI Exclusion Criteria (%)', 'Sustainable I

In [ ]:
'''
全ETF（US Listings）规模
'''

In [86]:
alletf.rename(columns={'Total Assets ':'Total Assets'})['Total Assets'].apply(lambda x: float(x[1:].replace(',',''))).sum()/10**12

3.883774553448